In [9]:
import os
from glob import glob

In [13]:
import numpy as np
np.set_printoptions(suppress=True)
from numpy import eye, dot
import cv2 as cv

In [1]:
import chessboard
import park_martin

In [2]:
from scipy.linalg import expm, inv

In [3]:
from tpk4170.hand_eye_calibration.utils import load_images, load_rob_poses

In [5]:
images = load_images()

In [7]:
rob_poses = load_rob_poses()

In [8]:
camera_matrix, dist_coeffs = chessboard.calibrate_lens(images)

In [9]:
def hat(v):
    return [[0, -v[2],  v[1]],
            [v[2],     0, -v[0]],
            [-v[1],  v[0],    0]]

In [14]:
def tf_mat(r, t):
    res = eye(4)
    res[0:3, 0:3] = expm(hat(r))
    res[0:3, -1] = t
    return res

In [16]:
obj_poses = []
corner_list = []
for i, img in enumerate(images):
    found, corners = chessboard.find_corners(img)
    corner_list.append(corners)
    if not found:
        raise Exception("Failed to find corners in img # %d" % i)
    rvec, tvec = chessboard.get_object_pose(
        chessboard.pattern_points, corners, camera_matrix, dist_coeffs)
    object_pose = tf_mat(rvec, tvec)
    obj_poses.append(object_pose)

In [18]:
A, B = [], []
for i in range(1, len(images)):
    p = rob_poses[i-1], obj_poses[i-1]
    n = rob_poses[i], obj_poses[i]
    A.append(dot(inv(p[0]), n[0]))
    B.append(dot(inv(p[1]), n[1]))

In [19]:
# Transformation to chessboard in robot gripper
X = eye(4)
Rx, tx = park_martin.calibrate(A, B)
X[0:3, 0:3] = Rx
X[0:3, -1] = tx

print("X: ")
print(X)

X: 
[[ 0.09853313 -0.99513261  0.00151965  0.04083886]
 [-0.99510923 -0.09854131 -0.00687275  0.09969052]
 [ 0.00698905 -0.00083503 -0.99997523  0.15899348]
 [ 0.          0.          0.          1.        ]]


In [23]:
print("For validation. Printing transformations from the robot base to the camera")
print("All the transformations should be quite similar")

for i in range(len(images)):
    rob = rob_poses[i]
    obj = obj_poses[i]
    tmp = dot(rob, dot(X, inv(obj)))
    print(tmp)

# Here I just pick one, but maybe some average can be used instead
rob = rob_poses[0]
obj = obj_poses[0]
cam_pose = dot(dot(rob, X), inv(obj))

For validation. Printing transformations from the robot base to the camera
All the transformations should be quite similar
[[ 0.73440851  0.01537859  0.67853332 -0.99273101]
 [-0.67862591  0.00112074  0.73448326 -0.8800224 ]
 [ 0.01053486 -0.99988113  0.01125937  0.74050302]
 [ 0.          0.          0.          1.        ]]
[[ 0.73262243  0.01627859  0.68044066 -0.99479051]
 [-0.6805475   0.00146293  0.7327025  -0.87803725]
 [ 0.01093194 -0.99986641  0.01215011  0.73824691]
 [ 0.          0.          0.          1.        ]]
[[ 0.73879692  0.01895382  0.67366148 -0.985715  ]
 [-0.67360731 -0.01006586  0.73902083 -0.87817649]
 [ 0.02078827 -0.99976965  0.0053308   0.75086777]
 [ 0.          0.          0.          1.        ]]
[[ 0.73959807  0.01997899  0.6727522  -0.98471032]
 [-0.67270204 -0.01013806  0.73984397 -0.87887138]
 [ 0.02160177 -0.99974908  0.00594186  0.75148219]
 [ 0.          0.          0.          1.        ]]
[[ 0.74415318  0.01941701  0.66772666 -0.97820253]
 [-0.6

# Hand-eye Calibration

Resource: 
- http://campar.in.tum.de/Chair/HandEyeCalibration 

## Eye-in-hand

![](https://www.torsteinmyhre.name/snippets/robcam_calibration/images/extrinsic-camera-calibration-eye-in-hand-651px.png)

## Eye-to-hand

![](https://www.torsteinmyhre.name/snippets/robcam_calibration/images/extrinsic-camera-calibration-stationary-camera-651px.png)